In [1]:
import pandas as pd 

import xml.etree.ElementTree as ET


tree = ET.parse("../data/raw/kml-poi.kml")
root = tree.getroot()

ns = {'kml': 'http://www.opengis.net/kml/2.2'}

placemarks = root.findall('.//kml:Placemark', ns)

if not placemarks:
    print(f"No Placemark elements found in '{kml_file}'.")

# Extract field names from the first Placemark with ExtendedData
field_names = []
first_placemark = next((pm for pm in placemarks if pm.find('./kml:ExtendedData/kml:SchemaData/kml:SimpleData', ns) is not None), None)


if first_placemark:
    simple_data_elements = first_placemark.findall('./kml:ExtendedData/kml:SchemaData/kml:SimpleData', ns)
    for element in simple_data_elements:
        name_attribute = element.get('name')
        if name_attribute and name_attribute not in field_names:
            field_names.append(name_attribute)
    if 'name' not in field_names and first_placemark.find('./kml:name', ns) is not None:
        field_names.insert(0, 'name')


data  = []
for placemark in placemarks:
    row_data = {}

    # Extract the name if present
    name_element = placemark.find('./kml:name', ns)
    if 'name' in field_names:
        row_data['name'] = name_element.text if name_element is not None else ""

    # Extract data from ExtendedData/SchemaData/SimpleData
    extended_data = placemark.find('./kml:ExtendedData/kml:SchemaData', ns)
    if extended_data is not None:
        simple_data_elements = extended_data.findall('./kml:SimpleData', ns)
        for element in simple_data_elements:
            name_attribute = element.get('name')
            if name_attribute in field_names:
                row_data[name_attribute] = element.text if element.text is not None else ""
    data.append(row_data)
    
# We just want "commissariat" located in Paris here 
df = pd.DataFrame(data).drop(["INFO_COMPLEMENTAIRES","NOM", "HORAIRES"], axis=1)
df["name"] = df.name.str.lower()
df_commissariat = df[df["name"].str.contains("commissariat")]
df_commissariat_paris = df_commissariat[df_commissariat["CODE_POSTA"].str.startswith("75")]
df_commissariat_paris["arrondissement"] = df_commissariat_paris["CODE_POSTA"].apply(lambda x : int(x[2:]))

df_commissariat_paris.to_csv("../data/commissariat_paris.csv")


C:\Users\Manon\AppData\Local\Temp\ipykernel_41728\3874235401.py:21: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if first_placemark:
C:\Users\Manon\AppData\Local\Temp\ipykernel_41728\3874235401.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_commissariat_paris["arrondissement"] = df_commissariat_paris["CODE_POSTA"].apply(lambda x : int(x[2:]))


In [2]:
df_commissariat_paris

,name,ADRESSE,CODE_POSTA,COMMUNE,arrondissement
17,commissariat,19/21 rue Truffaut,75017,Paris,17
19,commissariat central 1er,45 place du Marché Saint Honoré,75001,Paris,1
20,commissariat central 2e,18 rue du Croissant,75002,Paris,2
21,commissariat central 3e,4 bis - 6 rue aux Ours,75003,Paris,3
22,commissariat central 4e,27 boulevard Bourdon,75004,Paris,4
23,"commissariat central des 5e-6e, saip et bdep",4 rue de la Montagne Sainte Geneviève,75005,Paris,5
25,commissariat central 5e et 6e,"4, rue de la Montagne Sainte-Geneviève",75006,Paris,6
27,commissariat central 7e,9 rue Fabert,75007,Paris,7
29,commissariat central 8e,1 avenue du Général Eisenhower,75008,Paris,8
32,commissariat central 9e,14 bis rue Chauchat,75009,Paris,9
